In [ ]:

import sys, torch
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available(), "GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(i, torch.cuda.get_device_name(i))


In [ ]:

from tms_fractal_trainability import evaluate_grid, compose_rgb, save_rgb_image, select_device
import numpy as np

n, m = 256, 32       # features, embedding
S = 0.95             # uniform sparsity (P[x_i=0])
steps = 300
grid = (512, 512)

etaW_range = (1e-4, 1e+1)   # horizontal axis (η_W for W)
etaB_range = (1e-6, 1e+1)   # vertical axis (η_b for b)

device = select_device()
res = evaluate_grid(n=n, m=m, S=S, steps=steps,
                    etaW_range=etaW_range, etaB_range=etaB_range,
                    grid=grid, mode="analytic-L1",
                    log_etaW=True, log_etaB=True,
                    seed=123, dtype64=True, device=device,
                    N=8192, importance_uniform=True)

rgb = compose_rgb(res["diverged"], res["blue"], res["red"])
save_rgb_image("tms_preview.png", rgb, dpi=100)
print("Wrote tms_preview.png")


In [ ]:

from tms_fractal_trainability import evaluate_grid, compose_rgb, save_rgb_image, select_device

n, m = 256, 32
S = 0.95
steps = 300
grid = (256, 256)

res = evaluate_grid(n=n, m=m, S=S, steps=steps,
                    etaW_range=(1e-4, 5e-1), etaB_range=(1e-6, 1e-1),
                    grid=grid, mode="sampled", log_etaW=True, log_etaB=True,
                    seed=123, dtype64=True, device=select_device(),
                    N=16384, importance_uniform=True)

rgb = compose_rgb(res["diverged"], res["blue"], res["red"])
save_rgb_image("tms_preview_sampled.png", rgb, dpi=100)
print("Wrote tms_preview_sampled.png")



```bash
torchrun --standalone --nproc_per_node=8 tms_fractal_trainability.py \
  --ddp --mode analytic-L1 --n 256 --m 32 --S 0.95 --steps 500 \
  --etaW-min 1e-4 --etaW-max 1e+1 \
  --etaB-min 1e-6 --etaB-max 1e+1 \
  --width 4096 --height 4096 \
  --out tms_grid_4096x4096.npz
```


In [ ]:
from tms_fractal_trainability import compose_rgb, save_rgb_image
data = np.load("tms_grid_4096x4096.npz", allow_pickle=True)
rgb = compose_rgb(data["diverged"], data["blue"], data["red"])
save_rgb_image("tms_fractal_4096.png", rgb, dpi=100)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

data = np.load("tms_grid_4096x4096.npz", allow_pickle=True)  # or the preview .npz
for key, title in [("Dmean","Mean $D_i$"), ("Dstar","$D^* = m / \|W\|_F^2$")]:
    M = data[key]
    plt.figure(figsize=(6,6))
    im = plt.imshow(M, origin="lower", interpolation="nearest")
    plt.title(title)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()
